<a href="https://colab.research.google.com/github/sreenathk4589/END4NLP/blob/main/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input - 28x28x1 OUtput  28x28x8 RF 3
        # self.conv2 = nn.Conv2d(8, 8, 3, padding=1) #        28x28x8         28x28x8 RF 5
        # self.pool1 = nn.MaxPool2d(2, 2)            #        28x28x8         14x14x8 RF 7
        # self.conv3 = nn.Conv2d(8, 16, 3, padding=1)#        14x14x8         14x14x16 RF 
        # self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
        # self.pool2 = nn.MaxPool2d(2, 2)
        # self.conv5 = nn.Conv2d(16, 32, 3, padding=1)
        # self.conv6 = nn.Conv2d(32, 32, 3, padding=0)
        # self.conv7 = nn.Conv2d(32, 32, 1, padding=0)
        # nn.gap     = nn.AvgPool2d(3)


        self.convBlk1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.05))
        
        self.convBlk2 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.05))
        
        self.convBlk3 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 1, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 32, 1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(3))
        self.fc    = nn.Linear(32, 10)

    def forward(self, x):
        x = self.convBlk1(x)
        # print(x.size())
        x = self.convBlk2(x)
        # print(x.size())
        x = self.convBlk3(x)
        # print(x.size())
        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # # print("Blk1:\t")
        # # print(x.size())
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # # print("Blk2:\t")
        # # print(x.size())
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # # print("Blk3:\t")
        # # print(x.size())
        # x = F.relu(self.conv7(x))
        # # print("Blk4:\t")
        # # print(x.size())
        # x = F.relu(nn.gap(x));
        # # print("Blk5:\t")
        # # print(x.size())
        x = self.fc(x.view(x.size(0), -1));
        # print("Blk6:\t")
        # print(x.size())
        #x = x.view(-1, 10)
        #print(x.size())
        return F.log_softmax(x)

In [23]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4            [-1, 8, 28, 28]             584
              ReLU-5            [-1, 8, 28, 28]               0
       BatchNorm2d-6            [-1, 8, 28, 28]              16
         MaxPool2d-7            [-1, 8, 14, 14]               0
           Dropout-8            [-1, 8, 14, 14]               0
            Conv2d-9           [-1, 16, 14, 14]           1,168
             ReLU-10           [-1, 16, 14, 14]               0
      BatchNorm2d-11           [-1, 16, 14, 14]              32
           Conv2d-12           [-1, 16, 14, 14]           2,320
             ReLU-13           [-1, 16, 14, 14]               0
      BatchNorm2d-14           [-1, 16,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [24]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [25]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [27]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.058651555329561234 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.98it/s]



Test set: Average loss: 0.0712, Accuracy: 9788/10000 (98%)



loss=0.014520201832056046 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 21.24it/s]



Test set: Average loss: 0.0437, Accuracy: 9869/10000 (99%)



loss=0.022748390212655067 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.07it/s]


...


Test set: Average loss: 0.0216, Accuracy: 9935/10000 (99%)



loss=0.021663183346390724 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.44it/s]



Test set: Average loss: 0.0274, Accuracy: 9918/10000 (99%)



loss=0.0026526653673499823 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.45it/s]



Test set: Average loss: 0.0234, Accuracy: 9931/10000 (99%)

